# AutoGen ಮೂಲ ಮಾದರಿ

ಈ ಕೋಡ್ ಮಾದರಿಯಲ್ಲಿ, ನೀವು [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework ಅನ್ನು ಬಳಸಿಕೊಂಡು ಮೂಲ ಏಜೆಂಟ್ ಅನ್ನು ರಚಿಸುತ್ತೀರಿ.

ಈ ಮಾದರಿಯ ಉದ್ದೇಶವು ವಿವಿಧ ಏಜೆಂಟಿಕ್ ಮಾದರಿಗಳನ್ನು ಅನುಷ್ಠಾನಗೊಳಿಸುವಾಗ ನಾವು ನಂತರ ಬಳಸುವ ಹಂತಗಳನ್ನು ತೋರಿಸುವುದು.


## ಅಗತ್ಯವಿರುವ ಪೈಥಾನ್ ಪ್ಯಾಕೇಜ್‌ಗಳನ್ನು ಆಮದುಮಾಡಿ


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## ಕ್ಲೈಂಟ್ ಅನ್ನು ರಚಿಸಿ

ಈ ಮಾದರಿಯಲ್ಲಿ, ನಾವು [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ಅನ್ನು LLM ಗೆ ಪ್ರವೇಶಿಸಲು ಬಳಸುತ್ತೇವೆ.

`model` ಅನ್ನು `gpt-4o-mini` ಎಂದು ವ್ಯಾಖ್ಯಾನಿಸಲಾಗಿದೆ. GitHub Models ಮಾರುಕಟ್ಟೆಯಲ್ಲಿ ಲಭ್ಯವಿರುವ ಇನ್ನೊಂದು ಮಾದರಿಯನ್ನು ಪ್ರಯೋಗಿಸಲು ಮಾದರಿಯನ್ನು ಬದಲಾಯಿಸಿ ಮತ್ತು ವಿಭಿನ್ನ ಫಲಿತಾಂಶಗಳನ್ನು ನೋಡಿ.

ತ್ವರಿತ ಪರೀಕ್ಷೆಗಾಗಿ, ನಾವು ಸರಳ ಪ್ರಾಂಪ್ಟ್ ಅನ್ನು ನಡೆಸುತ್ತೇವೆ - `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ಏಜೆಂಟ್ ಅನ್ನು ನಿರ್ಧರಿಸುವುದು

ನಾವು ಈಗಾಗಲೇ `client` ಅನ್ನು ಸೆಟ್ ಅಪ್ ಮಾಡಿದ್ದು, ಅದು ಸರಿಯಾಗಿ ಕೆಲಸ ಮಾಡುತ್ತಿದೆ ಎಂದು ದೃಢಪಡಿಸಿದ್ದೇವೆ, ಈಗ `AssistantAgent` ಅನ್ನು ರಚಿಸೋಣ. ಪ್ರತಿ ಏಜೆಂಟ್‌ಗೆ ಕೆಳಗಿನವುಗಳನ್ನು ನಿಯೋಜಿಸಬಹುದು:  
**name** - ಬಹು ಏಜೆಂಟ್ ಪ್ರಕ್ರಿಯೆಗಳಲ್ಲಿ ಇದನ್ನು ಉಲ್ಲೇಖಿಸಲು ಉಪಯುಕ್ತವಾಗುವ ಚಿಕ್ಕ ಹೆಸರು.  
**model_client** - ನೀವು ಹಿಂದಿನ ಹಂತದಲ್ಲಿ ರಚಿಸಿದ ಕ್ಲೈಂಟ್.  
**tools** - ಏಜೆಂಟ್ ಕಾರ್ಯವನ್ನು ಪೂರ್ಣಗೊಳಿಸಲು ಬಳಸಬಹುದಾದ ಸಾಧನಗಳು.  
**system_message** - LLM ನ ಕಾರ್ಯ, ವರ್ತನೆ ಮತ್ತು ಶೈಲಿಯನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವ ಮೆಟಾಪ್ರಾಂಪ್ಟ್.  

ನೀವು system message ಅನ್ನು ಬದಲಾಯಿಸಿ LLM ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಬಹುದು. ನಾವು `tools` ಅನ್ನು ಪಾಠ #4 ನಲ್ಲಿ ಚರ್ಚಿಸುತ್ತೇವೆ.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸಿ

ಕೆಳಗಿನ ಫಂಕ್ಷನ್ ಏಜೆಂಟ್ ಅನ್ನು ಚಲಾಯಿಸುತ್ತದೆ. ನಾವು `on_message` ವಿಧಾನವನ್ನು ಬಳಸುತ್ತೇವೆ ಹೊಸ ಸಂದೇಶದೊಂದಿಗೆ ಏಜೆಂಟ್‌ನ ಸ್ಥಿತಿಯನ್ನು ನವೀಕರಿಸಲು.

ಈ ಸಂದರ್ಭದಲ್ಲಿ, ನಾವು ಬಳಕೆದಾರರಿಂದ ಹೊಸ ಸಂದೇಶವನ್ನು `"Plan me a great sunny vacation"` ಮೂಲಕ ಸ್ಥಿತಿಯನ್ನು ನವೀಕರಿಸುತ್ತೇವೆ.

ನೀವು ಸಂದೇಶದ ವಿಷಯವನ್ನು ಬದಲಾಯಿಸಿ LLM ಹೇಗೆ ವಿಭಿನ್ನವಾಗಿ ಪ್ರತಿಕ್ರಿಯಿಸುತ್ತದೆ ಎಂಬುದನ್ನು ನೋಡಬಹುದು.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕಾರ**:  
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ದಯವಿಟ್ಟು ಗಮನಿಸಿ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಡ್ಡೆಗಳು ಇರಬಹುದು. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜು ಪ್ರಾಮಾಣಿಕ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದವನ್ನು ಬಳಸುವ ಮೂಲಕ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಗಳ ಅಥವಾ ತಪ್ಪು ವ್ಯಾಖ್ಯಾನಗಳ ಬಗ್ಗೆ ನಾವು ಹೊಣೆಗಾರರಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
